# Normalize Audits vs Rejects EDA

Imports 

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel
from scipy import stats
import pyodbc
import matplotlib.pyplot as plt
import seaborn as sns

DB connonction details

In [ ]:
#redacted 

SQL

In [ ]:
audit_query = """
-- redacted 
"""

Joined_view = """
-- redacted 
"""

Initalize the Data Frames 

In [ ]:
with audit_conn:
    audit_df = pd.read_sql_query(audit_query, audit_conn)



with reject_conn:
    receipt_date_df = pd.read_sql_query(Joined_view, reject_conn)
    


audit_df.head()


In [ ]:
receipt_date_df.head(100)

we need to create a audit flag to do this we need to take the receipt_date_df and add a coluam called audit_last_90 that is a 1 or 0 flag column. that flag populats one if the [Supplier_Number] from receipt_date_df = [Supplier] from the audit df and the [audit conducted dt] from audit_df is with in 90 days of [Earliest_Rcvr_Dt] from receipt_date_df. so what i am looking to do is just see the rows from receipt_date_df that had a audit in the last 90 days. 

In [ ]:
#start funtions 

def flag_receipt_audit(
    receipt_df: pd.DataFrame,
    audit_df:   pd.DataFrame,
    # column names – change only if your source files use something else
    supplier_receipt_col: str = "supplier_no",          # column in receipt_df
    supplier_audit_col:   str = "Supplier",            # column in audit_df
    receipt_date_col:     str = "receipt_date",        # receipt datetime in receipt_df
    audit_date_col:       str = "audit conducted dt"   # audit datetime in audit_df
) -> pd.DataFrame:
    """
    Add a flag (`audit_last_90`) to `receipt_df` indicating whether the same
    supplier had at least one audit in the 90 days preceding each receipt.
    Returns a new DataFrame; the original is not modified.
    """

    #  flatten possible MultiIndex columns
    for df in (receipt_df, audit_df):
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = ["_".join(map(str, t)).strip("_") for t in df.columns]

    # remove exact duplicate rows 
    receipt_df = receipt_df.drop_duplicates().reset_index(drop=True)
    audit_df   = audit_df.drop_duplicates().reset_index(drop=True)

    # keep only needed columns and remember original row order 
    receipt_df = receipt_df.copy()
    receipt_df["_orig_idx"] = receipt_df.index                       # helper column
    jd = receipt_df[[supplier_receipt_col, receipt_date_col, "_orig_idx"]].copy()
    ad = audit_df[[supplier_audit_col, audit_date_col]].copy()

    # ensure dates are proper datetime objects
    jd[receipt_date_col] = pd.to_datetime(jd[receipt_date_col], errors="coerce")
    ad[audit_date_col]   = pd.to_datetime(ad[audit_date_col],   errors="coerce")

    #  drop rows where supplier or date could not be parsed 
    jd = jd.dropna(subset=[supplier_receipt_col, receipt_date_col])
    ad = ad.dropna(subset=[supplier_audit_col,   audit_date_col])

    # left‑join receipts to all audits for the same supplier 
    merged = jd.merge(
        ad,
        left_on=supplier_receipt_col,
        right_on=supplier_audit_col,
        how="left",
        suffixes=("_rcpt", "_audit")
    )

    # compute difference in days and flag audits within 0‑90 days 
    merged["day_diff"] = (merged[receipt_date_col] - merged[audit_date_col]).dt.days
    merged["audit_in_window"] = (
        (merged["day_diff"] >= 0) & (merged["day_diff"] <= 90)
    ).astype(int)

    # collapse back to one row per receipt (max flag per receipt) 
    flag_per_receipt = (
        merged.groupby("_orig_idx")["audit_in_window"]
              .max()
              .rename("audit_last_90")
    )

    # attach flag to the original receipt DataFrame 
    receipt_audit = receipt_df.copy()
    receipt_audit["audit_last_90"] = (
        flag_per_receipt.reindex(receipt_audit["_orig_idx"])
                       .fillna(0)
                       .astype(int)
    )
    receipt_audit = receipt_audit.drop(columns=["_orig_idx"])

    return receipt_audit



receipt_audit = flag_receipt_audit(
    receipt_df=receipt_date_df,
    audit_df=audit_df,
    supplier_receipt_col="supplier_no",
    supplier_audit_col="Supplier",
    receipt_date_col="receipt_date",
    audit_date_col="audit conducted dt"
)

# summary tables to chech distribution 
print("\n=== audit_last_90 flag distribution ===")
print(receipt_audit["audit_last_90"].value_counts())

In [ ]:
receipt_audit.head()

summary tables- now we need to combine all the flags we have set up. One thing to remeber is that when the flag for reject is found that means that no reject occored. Keep that in mind as we provide logic for the tables. 

In [ ]:



#create the 4 tables using logical tests 
df = receipt_audit.copy()

df["audit_non_rejects"]         = (df["audit_last_90"] == 1) & (df["reject_flag"] == 1)
df["audit_rejects"]     = (df["audit_last_90"] == 1) & (df["reject_flag"] == 0)
df["non_audit_non_rejects"]     = (df["audit_last_90"] == 0) & (df["reject_flag"] == 1)
df["non_audit_rejects"] = (df["audit_last_90"] == 0) & (df["reject_flag"] == 0)

# Convert the booleans to integers so they can be summed even though they shouls allready be integers
bool_cols = [
    "audit_rejects",
    "audit_non_rejects",
    "non_audit_rejects",
    "non_audit_non_rejects"
]
df[bool_cols] = df[bool_cols].astype(int)

# group by supplier and count flags 
Summary_counts = (
    df
    .groupby("supplier_no")[bool_cols]   # keep only the four columns we care about
    .sum()
    .reset_index()
)

#clean up names just to make sure i did not add spaces where i should not have
Summary_counts = Summary_counts.rename_axis(None, axis=1)


Summary_counts.head(10)

We only want to show rows where each of the counts is greater then 5. I did this to ensure that suppliers with low ammounts of receipts are not taken into account as small changes can have bigger impacts of the ratios later down the road. 

In [ ]:
# filter to only show rows that dont have 0s in the count fields 
count_cols = [
    "audit_rejects",
    "audit_non_rejects",
    "non_audit_rejects",
    "non_audit_non_rejects"
]

filtered_SC = (
    Summary_counts[ (Summary_counts[count_cols] > 0).all(axis=1) ]
                .reset_index(drop=True)      # clean index
)


filtered_SC.head(10)

Here we create the ratios of rejects for both audited and non audited reciepts. 

In [ ]:


#  Build the ratio DataFrame

filtered_ratio_SC = (
    filtered_SC[['supplier_no']]                             
    .assign(
        audit_ratio = filtered_SC['audit_non_rejects'] / filtered_SC['audit_rejects'],
        non_audit_ratio = filtered_SC['non_audit_non_rejects'] / filtered_SC['non_audit_rejects']
    )
)

# clean up any infinite or NaN values that arise from a 0 divisor

filtered_ratio_SC['audit_ratio'] = filtered_ratio_SC['audit_ratio'].replace([float('inf'), -float('inf')], pd.NA)
filtered_ratio_SC['non_audit_ratio'] = filtered_ratio_SC['non_audit_ratio'].replace([float('inf'), -float('inf')], pd.NA)

filtered_ratio_SC.head(10)

Here we run the t test to see the avrage diffrence between the 2 values, the p test to tell if the diffrance is statisticaly significant and the Cohen's d (paired) test to determine the level of magnitude in diffracne. 

In [ ]:

audit_vals     = filtered_ratio_SC['audit_ratio'].values
non_audit_vals = filtered_ratio_SC['non_audit_ratio'].values

# paired (“related samples”) t‑test
t_stat, p_val = stats.ttest_rel(non_audit_vals, audit_vals, nan_policy='omit')

print('--- Paired t‑test results ---')
print(f't‑statistic : {t_stat:.4f}')
print(f'p‑value    : {p_val:.4g}')


# effect size (Cohen’s d for paired data)

diff = non_audit_vals - audit_vals
cohen_d = diff.mean() / diff.std(ddof=1)  
print(f"Cohen's d (paired) : {cohen_d:.3f}")

# lets look for outliers with a quick diffrance chart

import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(6,4))
sns.histplot(diff, kde=True, bins=15, color='steelblue')
plt.title('Distribution of (non_audit_ratio – audit_ratio)')
plt.xlabel('Difference')
plt.show()

We saw a few very extreme values in the difference between non‑audit and audit ratios that were making a tiny but unrealistic statistical significance look like audits caused more rejects. To fix this we applied the standard box‑plot rule: we calculated the inter‑quartile range, set the lower and upper limits at Q1 minus 1.5 times the IQR and Q3 plus 1.5 times the IQR, and dropped any points outside those limits.

In [ ]:


# 1. Compute the difference
diff = filtered_ratio_SC['non_audit_ratio'] - filtered_ratio_SC['audit_ratio']

# 2. Find IQR and whisker fences
q1 = diff.quantile(0.25)
q3 = diff.quantile(0.75)
iqr = q3 - q1
lower_fence = q1 - 1.5 * iqr
upper_fence = q3 + 1.5 * iqr

# 3. Keep only non‑outlier rows (mask stays aligned for both ratios)
mask = (diff >= lower_fence) & (diff <= upper_fence)
audit_clean     = filtered_ratio_SC.loc[mask, 'audit_ratio'].values
non_audit_clean = filtered_ratio_SC.loc[mask, 'non_audit_ratio'].values
diff_clean      = diff[mask].values

# 4. Paired t‑test & Cohen's d on the cleaned data
t_stat, p_val = stats.ttest_rel(non_audit_clean, audit_clean, nan_policy='omit')
cohen_d = diff_clean.mean() / diff_clean.std(ddof=1)

print('t‑statistic :', round(t_stat, 4))
print('p‑value    :', round(p_val, 4))
print("Cohen's d  :", round(cohen_d, 3))

# 5. Visualise the cleaned distribution
plt.figure(figsize=(6, 4))
sns.histplot(diff_clean, kde=True, bins=15, color='steelblue')
plt.title('Difference after outlier removal')
plt.xlabel('non_audit_ratio - audit_ratio')
plt.show()

From this analysis we see that the difference in ratios is not significantly different, and therefore there is no reason to think that audits have an effect on rejects once we normalize to receipt.